In [ ]:
• preprocessing.py
# 전처리
import pandas as pd 
import numpy as np 
from scipy.stats import chi2

## 이상치 제거
def mahalanobis(x, data):
    x_minus_mu = x - np.mean(data, axis=0)
    cov=np.cov(data,rowvar=False)
    inv_covmat = np.linalg.inv(cov)
    left_term = np.dot(x_minus_mu, inv_comat)
    mahal = np.dot(left_term, x_minus_mu.T)
    return mahal.diagonal()
def outlier_detection(df):
    range_q1, range_q3 = np.percentile(df['거리오차'],[25,75])
    def_q1, def_q3 = np.percentile(df['표준편의'],[25,75])
    range_iqr = range_q3 - range_q1
    def_iqr = def_q3 - def_q1
    range_lower = range_q1 - (range_iqr*1.5)
    range_upper = range_q3 + (range_iqr*1.5)
    def_lower = def_q1 - (def_iqr*1.5)
    def_upper = def_q3 + (def_iqr*1.5)
    
    df_x = df[['거리오차','표준편의']].reset_index(drop=True)
    
    df_x['mahala'] = mahalanobis(x=df_x, data=df_x)
    df_x['p_value'] = 1 - chi2.cdf(df_x['mahala'],1),
    df_x['Outlier_maha'] = ["outlier" if x < 0.01 else "normal" for x in df_x['p_value']]
    df_x['거리오차_이상치']=["outlier" if (x< range_lower) or (x> range_upper) else "normal" for x in df_x['거리오차']]
    df_x['표준편의_이상치']=["outlier" if (x< def_lower) or (x> def_upper) else "normal" for x in df_x['표준편의']]

    return df_x[['거리오차','표준편의','거리오차_이상치','표준편의_이상치','Outlier_maha']]
                 
def outlier(x):
    temp_df = x[['거리오차', '표준편의']]
    temp_df_outlier = outlier_detection(temp_df)
    x = x.iloc[temp_df_outlier[(temp_df_outlier['Outlier_maha']=='normal')].index,:]
    × = x.drop('편의오차',axis=1)
    x = x.rename(columns={'표준편의 : 편의오차'})
    return x

## 풍향 숫자화
def EtoN(X):
    if isinstance(x, str):
        char_mapping = {'N': 22.4, 'NE' : 67.4, 'E' : 112.4, 'SE': 157.4, 'S':202.4,'SW':247.4, 'W':292.4, 'NW' :337.4} 
        return char_mapping.get(x,x)
    else:
        return x

# 데이터는 비문처리(군사자료는 비문!)
df = pd.read_csv('함포.csv',index_col=None)

# 격파사격(SALVO)만 사용
df=df[df['사격종류']=='RF']
df=df[df['사격방법']=='SALVO']
a=df['편의오차'].astype(str)
b=df['거리오차']

# 편의, 거리오차 전처리
for i in range(len(a)):
    if '~' in a.iloc[i]:
        a.iloc[i]='9999'
    if a.iloc[i]=='-':
        a.iloc[i]='9999'
a=a.replace(to_replace=r'R', value='',regex=True)
a=a.replace(to_replace=r'L', value='-', regex=True)
a=a.replace(to_replace=r'HIT', value='0', regex=True) 
a=a.replace(to_replace=r'H', value='0', regex=True)
a=a replace(to_replace=r'관측불가', value='9999', regex=True)

for i in range(len(b)):
    if b.iloc[i]=='-':
        b.iloc[i]='9999'
b=b.replace(to_replace=r',', value='', regex=True)
b=b.replace(to_replace=r'Hit', value='0', regex=True)
b=b.replace(to_replace=r'HIT', value='0', regex=True)
b=b.replace(to_replace=r'H', value='0', regex=True)
b=b.replace(to_replace=r'관측불가', value='9999', regex=True)
a.astype(float) 
b.astype(float)
df['편의오차']=a 
df['거리오차']=b 

# 사격거리 결측치 제거
df=df[df['사격거리'].isna()!=True]
# 풍향속 전처리
df[['풍향','풍속']]=df['풍향속'].str.split('-', expand=True)
df=df.drop(['풍향속'],axis= 1)
df=df[df['풍향']!='NW/NE']
df['풍향']=df['풍향'].map(EtoN)
df['풍향']=df['풍향'].astype(float)

# 파고 전처리
df=df[(df['파고']!='0.5~1') & (df['파고']!='1~2') & (df['파고']!='1~1.5') & (df['파고']!='1.5~2') & (df['파고']!='1.4') & (df['파고']!='0.2') & (df['파고']!='-')] 

# 변수 선정 및 전처리
df=df[['연 도','월 일','함 형','함포종류','풍향','풍속','파 고','사격거리','거리오차','편의오차','지정사거리']]
df[['거리오차','편의오차','사격거리']] = df[['거리오차''편의오차','사격거리']].astype('float')
df['풍속']= df['풍속'].astype('float')
df = df[(df['풍속']<=15)]
df127 = df[(df['함포종류']=='5inch')]
df76 = df[(df['함포종류']=='76mm')]
df40 = df[(df['함포종류']=='40mm')]
df = pd.concat([df127,df76,df40],axis=0)

# 이상치 제거
df = df[(df['거리오차']!=9999) & (df['편의오차']!=9999)]
# 표준편의 변환
df['표준편의']= round(df['지정사거리']/df['사격거리']*df['편의오차'],2)
df127 = df[df['함포종류']=='5inch']
df76 = df[df['함포종류']=='76mm']
df40 = df[df['함포종류']=='40mm']

# 표준편의 기준 결측치 제거
df40 = df40.dropna(subset=['표준편의'])

# 이상치 제거
df127 = outlier(df127)
df40 = outlier(df40)
df76 = outlier(df76)

# 127mm, 76mm 데이터 결합
df127['거리오차'] = df127['거리오차'].astype('float')
df76['거리오차']= df76['거리오차'].astype('float')
df = pd.concat([df127,df76], axis=0)

# 함형별 묶기
df_all = df[(df['함 형']=='DDH')|(df['함 형']=='FFG')|(df['함 형']=='PCC')|(df['함 형']=='PKG')]
df_40 = df40[df40['함 형']=='PKM'] 

# 함형별 데이터, 종합
df = pd.concat([df_all, df_40],axis=0)

# 전달정확도 데이터 2가지로 나눠서 구성 (MPI : df1, PREC : df2)
df[['거리오차','편의오차','사격거리']] = df[['거리오차', '편의오차' '사격거리']].astype('float')
df1 = df.groupby(['연 도','월 일','함 형', '함포종류','풍향','풍속','파 고'])[['거리오차','편의오차']].mean()
df1 = df1.rename(columns = {'거리오차':'MPI_Y','편의오차':'MPI_X'})
df2 = df.groupby(['연 도','월 일','함 형', '함포종류','풍향','풍속','파 고'][['거리오차','편의오차']].std()
df2 = df2.rename(columns = {'거리오차':'PREC_Y','편의오차':'PREC_X'})
df1.reset_index(inplace=True)
df2.reset_index(inplace=True)

# 데이터 종합 후 전처리
df1 = df1[['MPI_Y','MPI_X']]
DA = pd.concat([df1,df2], axis=1)
DA = DA.dropna()
                 
df1 = DA[['함 형','풍향','풍속','파 고','MPI_Y', 'MPI_X']]
df2 = DA[['함 형','풍향','풍속','파고','PREC_Y','PREC_X']]

df1 = df1.rename(columns= {'MPI_Y':'거리오차', 'MPI_X':'편의오차'})
df2 = df2.rename(columns= {'PREC_Y':'거리오차', 'PREC_X':'편의오차'}) 

# 최종 Feature 선정
df1 = df1[['함 형','풍향','풍속','파 고','거리오차']]
df2 = df2[['함 형','풍향','풍속','파 고','편의오차']]

# 전달정확도 데이터셋(MPI_거리,MPL편의, PREC_거리, PREC_편의)
df_y1 = df1[['함 형','풍향','풍속','파 고','거리오차']]
df_x1 = df1[['함 형','풍향','풍속','파 고','편의오차']]
df_y2 = df2[['함 형','풍향','풍속','파 고','거리오차']]
df_x2 = df2[['함 형','풍향','풍속','파 고','편의오차']]
                 
# 전달정확도 데이터 저장
df_y1.to_csv('MPI_거리_raw.csv',index=False)
df_x1.to_csv('MPI_편의_raw.csv',index=False)
df_y2.to_csv('PREC_거리_raw.csv',index=False)
df_x2.to_csv('PREC_편의_raw.csv',index=False)
print('전처리 완료!')


In [ ]:
• main.py
import pandas as pd 
import os, random 
import numpy as np 
import torch

from ctgan 
import CTGAN from sklearn.cluster 
import KMeans from statsmodels.stats.outliers_influence 
import variance_inflation_factor from autogluon.tabular 
import TabularPredictor 
import time

# 전처리 시작
def main():
    print("데이터를 선택합니다.")
    print("원하시는 번호를 입력해주세요.WnWn")
    print("1. MPI_거리")
    print("2. MPL편의")
    print("3. PREC_거리")
    print("4. PREC_편의")
    key= int(input("\n\n숫자만 입력하세요: "))
    
    switch_dict={
        1:"MPI_거리",
        2:"MPI_편의",
        3:"PREC_거리",
        4:"PREC_편의",
    }
    result = switch_dict.get(key,"잘못 입력하셨습니다.")
    print(f"\n\n{key}. {result}를 선택하셨습니다.")
    return key, result
# 회귀 시작
def main2():
    print("데이터의 종류를 선택합니다.")
    print("\n\n원하시는 번호를 입력해주세요.")
    print("1. RAW")
    print("2. AUG")
    print("3. 종료")
    key= int(input("\n\n숫자만 입력하세요: "))
    switch_dict={
        1: "df_raw",
        2: "df_all",
        3:'회귀 종료'
    }
    result = switch_dict.get(key, "잘못 입력하셨습니다.")
    if key ==1 or key==2:
        print(f"\n\n{result}의 성능평가를 시작합니다.")
    else:
        print("\n\n종료합니다.")
    return key

# 시드고정
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark=False 
    torch.use_deterministic_algorithms(True)
    os.environ["PYTHONHASHSEED"] =str(seed)
    os.environ["TF_ENABLE_ONEDNN_OPTS"]="0"
# 풍향 군집화
def WD(x):
    kmeans = KMeans(n_clusters=8, random_state=42)
    pred = kmeans.fit_predict(x.reshape(-1,1))
    ndf['풍향']=pred
    df_wd = pd.get_dummies(ndf['풍향'], prefix='풍향')
    wd_list=['풍향_O', '풍향_1', '풍향_2', '풍향_3', '풍향_4', '풍향_5', '풍향_6', '풍향_7']
    return df_wd, wd_list

# 거리오차
def REP(rep, key):
    if key==1:
        n=8
        r_list= ['거리_O', '거리_1', '거리_2', '거리_3','거리_4','거리_5', '거리_6', '거리_7']
    else :
        n=10
        r_list=['거리_O', '거리_1', '거리_2', '거리_3', '거리_4','거리_5','거리_6','거리_7','거리_8','거리_9']
        kmeans = KMeans(n_clusters=n, random_state=42)
        pred = kmeans.fit_predict(rep.reshape(-1,1))
        ndf['거리오차']=pred
        rep = pd.get_dummies(ndf['거리오차'], prefix='거리')
        df_c=['DDH', 'FFG', 'PCC', 'PKG', 'PKM', '풍속', '풍향_0','풍향_1', '풍향_2', '풍향_3', '풍향_4','풍향_5', '풍향_6', '풍향_7', '파 고', 'ER', '거리오차']
        return rep, _list, df_c, n

# 편의오차
def DEP(dep, key):
    if key==2:
        n=8
        r_list=['편의_0', '편의_1', '편의_2', '편의_3', '편의_4', '편의_5', '편의_6', '편의_7']
    else :
        n=8
        r_list=['편의_O', '편의_1', '편의_2', '편의_3', '편의_4', '편의_5', '편의_6', '편의_7']
        kmeans = KMeans(n_clusters=n, random_state=42)
        pred = kmeans.fit_predict(dep.reshape(-1,1))
        ndf['편의오차']=pred
        dep = pd.get_dummies(ndf['편의오차'], prefix='편의')
        df_c=['DDH', 'FFG', 'PCC', 'PKG', 'PKM', '풍속', '풍향_0', '풍향_1', '풍향_2', '풍향_3', '풍향 4', '풍향_5', '풍향_6', '풍향_7', '파 고', 'ER', '편의오차']
        return dep, r_list, df_c, n
# 풍향 개수 맞추기
def wd_match(key):
    # MPI_거리
    wd_num=51
    if key == 1:
        m=df_raw['풍향_0'].value_counts().values.tolist()
        df_O = df_aug[(df_aug['wd']=='풍향_0')].sample(n=wd_num-m[1],replace=True,random_state=42)
        m=df_raw['풍향_1'].value_counts().values.tolist()
        df_1 = df_aug[(df_aug['wd']=='풍향_1')].sample(n=wd_num-m[1],replace=True,random_state=42)
        m=df_raw['풍향_2'].value_counts().values.tolist()
        df_2 = df_aug[(df_aug['wd']=='풍향_2')].sample(n=wd_num-m[1],replace=True,random_state=42)
        m=df_raw['풍향_3'].value_counts().values.tolist()
        df_3 = df_aug[(df_aug['wd']=='풍향_3')].sample(n=wd_num-m[1],replace=True,random_state=42)
        m=df_raw['풍향_4'].value_counts().values.tolist()
        df_4 = df_aug[(df_aug['wd']=='풍향_4')].sample(n=wd_num-m[1],replace=True,random_state=42)
        m=df_raw['풍향_5'].value_counts().values.tolist()
        df_5 = df_aug[(df_aug['wd']=='풍향_5')].sample(n=wd_num-m[1],replace=True,random_state=42)
        m=df_raw['풍향_6'].value_counts().values.tolist()
        df_6 = df_aug[(df_aug['wd']=='풍향_6')].sample(n=wd_num-m[1],replace=True,random_state=42)
        m=df_raw['풍향_7'].value_counts().values.tolist()
        df_7 = df_aug[(df_aug['wd']=='풍향_7')].sample(n=wd_num-m[1],replace=True,random_state=42)
    # MPI_편의
    elif key == 2:
        m=df_raw['풍향_0'].value_counts().values.tolist()
        df_O = df_aug[(df_aug['wd']=='풍향_0')].sample(n=wd_num-m[1],replace=True,random_state=42)
        m=df_raw['풍향_1'].value_counts().values.tolist()
        df_1 = df_aug[(df_aug['wd']=='풍향_1')].sample(n=wd_num-m[1],replace=True,random_state=42)
        m=df_raw['풍향_2'].value_counts().values.tolist()
        df_2 = df_aug[(df_aug['wd']=='풍향_2')].sample(n=wd_num-m[1],replace=True,random_state=42)
        m=df_raw['풍향_3'].value_counts().values.tolist()
        df_3 = df_aug[(df_aug['wd']=='풍향_3')].sample(n=wd_num-m[1],replace=True,random_state=42)
        m=df_raw['풍향_4'].value_counts().values.tolist()
        df_4 = df_aug[(df_aug['wd']=='풍향_4')].sample(n=wd_num-m[1],replace=True,random_state=42)
        m=df_raw['풍향_5'].value_counts().values.tolist()
        df_5 = df_aug[(df_aug['wd']=='풍향_5')].sample(n=wd_num-m[1],replace=True,random_state=42)
        m=df_raw['풍향_6'].value_counts().values.tolist()
        df_6 = df_aug[(df_aug['wd']=='풍향_6')].sample(n=wd_num-m[1],replace=True,random_state=42)
        m=df_raw['풍향_7'].value_counts().values.tolist()
        df_7 = df_aug[(df_aug['wd']=='풍향_7')].sample(n=wd_num-m[1],replace=True,random_state=42)
    # PREC_거리
    elif key == 3:
        m=df_raw['풍향_0'].value_counts().values.tolist()
        df_O = df_aug[(df_aug['wd']=='풍향_0')].sample(n=wd_num-m[1],replace=True,random_state=42)
        m=df_raw['풍향_1'].value_counts().values.tolist()
        df_1 = df_aug[(df_aug['wd']=='풍향_1')].sample(n=wd_num-m[1],replace=True,random_state=42)
        m=df_raw['풍향_2'].value_counts().values.tolist()
        df_2 = df_aug[(df_aug['wd']=='풍향_2')].sample(n=wd_num-m[1],replace=True,random_state=42)
        m=df_raw['풍향_3'].value_counts().values.tolist()
        df_3 = df_aug[(df_aug['wd']=='풍향_3')].sample(n=wd_num-m[1],replace=True,random_state=42)
        m=df_raw['풍향_4'].value_counts().values.tolist()
        df_4 = df_aug[(df_aug['wd']=='풍향_4')].sample(n=wd_num-m[1],replace=True,random_state=42)
        m=df_raw['풍향_5'].value_counts().values.tolist()
        df_5 = df_aug[(df_aug['wd']=='풍향_5')].sample(n=wd_num-m[1],replace=True,random_state=42)
        m=df_raw['풍향_6'].value_counts().values.tolist()
        df_6 = df_aug[(df_aug['wd']=='풍향_6')].sample(n=wd_num-m[1],replace=True,random_state=42)
        m=df_raw['풍향_7'].value_counts().values.tolist()
        df_7 = df_aug[(df_aug['wd']=='풍향_7')].sample(n=wd_num-m[1],replace=True,random_state=42)
    # PREC_편의
    else :
        m=df_raw['풍향_0'].value_counts().values.tolist()
        df_O = df_aug[(df_aug['wd']=='풍향_0')].sample(n=wd_num-m[1],replace=True,random_state=42)
        m=df_raw['풍향_1'].value_counts().values.tolist()
        df_1 = df_aug[(df_aug['wd']=='풍향_1')].sample(n=wd_num-m[1],replace=True,random_state=42)
        m=df_raw['풍향_2'].value_counts().values.tolist()
        df_2 = df_aug[(df_aug['wd']=='풍향_2')].sample(n=wd_num-m[1],replace=True,random_state=42)
        m=df_raw['풍향_3'].value_counts().values.tolist()
        df_3 = df_aug[(df_aug['wd']=='풍향_3')].sample(n=wd_num-m[1],replace=True,random_state=42)
        m=df_raw['풍향_4'].value_counts().values.tolist()
        df_4 = df_aug[(df_aug['wd']=='풍향_4')].sample(n=wd_num-m[1],replace=True,random_state=42)
        m=df_raw['풍향_5'].value_counts().values.tolist()
        df_5 = df_aug[(df_aug['wd']=='풍향_5')].sample(n=wd_num-m[1],replace=True,random_state=42)
        m=df_raw['풍향_6'].value_counts().values.tolist()
        df_6 = df_aug[(df_aug['wd']=='풍향_6')].sample(n=wd_num-m[1],replace=True,random_state=42)
        m=df_raw['풍향_7'].value_counts().values.tolist()
        df_7 = df_aug[(df_aug['wd']=='풍향_7')].sample(n=wd_num-m[1],replace=True,random_state=42)
    # RAW대비 개수 맞춰준 데이터 생성
    df_a= pd.concat([df_0,df_1,df_2,df_3,df_4,df_5,df_6,df_7], axis=0)
    return df_a

## 산출물 함수
def ST(x):
    if x==0 :
        st= pd.DataFrame({'DDH': [0],'FFG' :[0], 'PCC' : [0], 'PKG': [0], 'PKM': [1]})
    elif x==1 :
        st= pd.DataFrame({'DDH': [0],'FFG' :[0], 'PCC' : [0], 'PKG': [1], 'PKM': [0]})
    elif x==2 :
        st= pd.DataFrame({'DDH': [0],'FFG' :[0], 'PCC' : [1], 'PKG': [0], 'PKM': [0]})
    elif x==3 :
        st= pd.DataFrame({'DDH': [0],'FFG' :[1], 'PCC' : [0], 'PKG': [0], 'PKM': [0]})
    else :
        st= pd.DataFrame({'DDH': [1],'FFG' :[0], 'PCC' : [0], 'PKG': [1], 'PKM': [0]})
    return st
# 풍속
def WS(x):
    if x==0:
        ws= pd.DataFrame({'풍속':[5]})
    elif x==1:
        ws= pd.DataFrame({'풍속':[8]})
    elif x==2:
        ws= pd.DataFrame({'풍속':[10]})
    return ws
# 파고
def WV(x):
    if x==0:
        wv= pd.DataFrame({'파 고':[0.5]})
    elif x==1:
        wv= pd.DataFrame({'파 고':[1.0]})
    else:
        wv= pd.DataFrame({'파 고':[1.5]})
    return wv
# 풍향
def WDD(x):
    if x==2:
        wd= pd.DataFrame({'풍향_0':[0], '풍향_1':[0], '풍향_2':[1], '풍향_3' :[0], '풍향_4':[0], '풍향_5' :[0], '풍향_6' :[0], '풍향_7' :[0]})
    elif x==4:
        wd= pd.DataFrame({'풍향_0':[0], '풍향_1':[0], '풍향_2':[0], '풍향_3' :[0], '풍향_4':[1], '풍향_5' :[0], '풍향_6' :[0], '풍향_7' :[0]})
    elif x==7:
        wd= pd.DataFrame({'풍향_0':[0], '풍향_1':[0], '풍향_2':[0], '풍향_3' :[0], '풍향_4':[0], '풍향_5' :[0], '풍향_6' :[0], '풍향_7' :[1]})
    elif x==0:
        wd= pd.DataFrame({'풍향_0':[1], '풍향_1':[0], '풍향_2':[1], '풍향_3' :[0], '풍향_4':[0], '풍향_5' :[0], '풍향_6' :[0], '풍향_7' :[0]})
    elif x==5:
        wd= pd.DataFrame({'풍향_0':[0], '풍향_1':[0], '풍향_2':[0], '풍향_3' :[0], '풍향_4':[0], '풍향_5' :[1], '풍향_6' :[0], '풍향_7' :[0]})
    elif x==3:
        wd= pd.DataFrame({'풍향_0':[0], '풍향_1':[0], '풍향_2':[0], '풍향_3' :[1], '풍향_4':[0], '풍향_5' :[0], '풍향_6' :[0], '풍향_7' :[0]})
    elif x==6:
        wd= pd.DataFrame({'풍향_0':[0], '풍향_1':[0], '풍향_2':[0], '풍향_3' :[0], '풍향_4':[0], '풍향_5' :[0], '풍향_6' :[1], '풍향_7' :[0]})
    else:
        wd= pd.DataFrame({'풍향_0':[0], '풍향_1':[1], '풍향_2':[0], '풍향_3' :[0], '풍향_4':[0], '풍향_5' :[0], '풍향_6' :[0], '풍향_7' :[0]})
    return wd
# 오차범위
def DA(x, j): 
    global da, df_all
    if x=='MPI_거리':
        for i in range(j):
            if i==0:
                da=pd.DataFrame({f'{x[-2:]}_O' : [1], f'{x[-2:]_1' : [0], f'{x[-2:]_2' : [0], f'{x[-2:]_3' : [0], f'{x[-2:]_4' : [0], f'{x[-2:]_5': [0], f'{x[-2:]}_6': [0], f'{x[-2:]_7': [0]})
            elif i==1:
                da=pd.DataFrame({f'{x[-2:]}_O' : [0], f'{x[-2:]_1' : [1], f'{x[-2:]_2' : [0], f'{x[-2:]_3' : [0], f'{x[-2:]_4' : [0], f'{x[-2:]_5': [0], f'{x[-2:]}_6': [0], f'{x[-2:]_7': [0]})
            elif i==2:
                da=pd.DataFrame({f'{x[-2:]}_O' : [0], f'{x[-2:]_1' : [0], f'{x[-2:]_2' : [1], f'{x[-2:]_3' : [0], f'{x[-2:]_4' : [0], f'{x[-2:]_5': [0], f'{x[-2:]}_6': [0], f'{x[-2:]_7': [0]})
            elif i==3:
                da=pd.DataFrame({f'{x[-2:]}_O' : [0], f'{x[-2:]_1' : [0], f'{x[-2:]_2' : [0], f'{x[-2:]_3' : [1], f'{x[-2:]_4' : [0], f'{x[-2:]_5': [0], f'{x[-2:]}_6': [0], f'{x[-2:]_7': [0]})
            elif i==4:
                da=pd.DataFrame({f'{x[-2:]}_O' : [0], f'{x[-2:]_1' : [0], f'{x[-2:]_2' : [0], f'{x[-2:]_3' : [0], f'{x[-2:]_4' : [1], f'{x[-2:]_5': [0], f'{x[-2:]}_6': [0], f'{x[-2:]_7': [0]})
            elif i==5:
                da=pd.DataFrame({f'{x[-2:]}_O' : [0], f'{x[-2:]_1' : [0], f'{x[-2:]_2' : [0], f'{x[-2:]_3' : [0], f'{x[-2:]_4' : [0], f'{x[-2:]_5': [1], f'{x[-2:]}_6': [0], f'{x[-2:]_7': [0]})
            elif i==6:
                da=pd.DataFrame({f'{x[-2:]}_O' : [0], f'{x[-2:]_1' : [0], f'{x[-2:]_2' : [0], f'{x[-2:]_3' : [0], f'{x[-2:]_4' : [0], f'{x[-2:]_5': [0], f'{x[-2:]}_6': [1], f'{x[-2:]_7': [0]})
            elif i==7:
                da=pd.DataFrame({f'{x[-2:]}_O' : [0], f'{x[-2:]_1' : [0], f'{x[-2:]_2' : [0], f'{x[-2:]_3' : [0], f'{x[-2:]_4' : [0], f'{x[-2:]_5': [0], f'{x[-2:]}_6': [0], f'{x[-2:]_7': [1]})
            max = da.idxmax(axis=1)
            e_r=pd.DataFrame()
            e_r['ER']=max
            da=e_r['ER']
            new_row = pd.concat([gt,wd,ws,wv,da], axis=1)
            df_all=pd.concat([df_all,pd.DataFrame(new_row, index=[0])])
    elif x == 'MPI_편의':
        for i in range(j):
            if i==0:
                da=pd.DataFrame({f'{x[-2:]}_O' : [1], f'{x[-2:]_1' : [0], f'{x[-2:]_2' : [0], f'{x[-2:]_3' : [0], f'{x[-2:]_4' : [0], f'{x[-2:]_5': [0], f'{x[-2:]}_6': [0], f'{x[-2:]_7': [0]})
            elif i==1:
                da=pd.DataFrame({f'{x[-2:]}_O' : [0], f'{x[-2:]_1' : [1], f'{x[-2:]_2' : [0], f'{x[-2:]_3' : [0], f'{x[-2:]_4' : [0], f'{x[-2:]_5': [0], f'{x[-2:]}_6': [0], f'{x[-2:]_7': [0]})
            elif i==2:
                da=pd.DataFrame({f'{x[-2:]}_O' : [0], f'{x[-2:]_1' : [0], f'{x[-2:]_2' : [1], f'{x[-2:]_3' : [0], f'{x[-2:]_4' : [0], f'{x[-2:]_5': [0], f'{x[-2:]}_6': [0], f'{x[-2:]_7': [0]})
            elif i==3:
                da=pd.DataFrame({f'{x[-2:]}_O' : [0], f'{x[-2:]_1' : [0], f'{x[-2:]_2' : [0], f'{x[-2:]_3' : [1], f'{x[-2:]_4' : [0], f'{x[-2:]_5': [0], f'{x[-2:]}_6': [0], f'{x[-2:]_7': [0]})
            elif i==4:
                da=pd.DataFrame({f'{x[-2:]}_O' : [0], f'{x[-2:]_1' : [0], f'{x[-2:]_2' : [0], f'{x[-2:]_3' : [0], f'{x[-2:]_4' : [1], f'{x[-2:]_5': [0], f'{x[-2:]}_6': [0], f'{x[-2:]_7': [0]})
            elif i==5:
                da=pd.DataFrame({f'{x[-2:]}_O' : [0], f'{x[-2:]_1' : [0], f'{x[-2:]_2' : [0], f'{x[-2:]_3' : [0], f'{x[-2:]_4' : [0], f'{x[-2:]_5': [1], f'{x[-2:]}_6': [0], f'{x[-2:]_7': [0]})
            elif i==6:
                da=pd.DataFrame({f'{x[-2:]}_O' : [0], f'{x[-2:]_1' : [0], f'{x[-2:]_2' : [0], f'{x[-2:]_3' : [0], f'{x[-2:]_4' : [0], f'{x[-2:]_5': [0], f'{x[-2:]}_6': [1], f'{x[-2:]_7': [0]})
            elif i==7:
                da=pd.DataFrame({f'{x[-2:]}_O' : [0], f'{x[-2:]_1' : [0], f'{x[-2:]_2' : [0], f'{x[-2:]_3' : [0], f'{x[-2:]_4' : [0], f'{x[-2:]_5': [0], f'{x[-2:]}_6': [0], f'{x[-2:]_7': [1]})
            max = da.idxmax(axis=1)
            e_r=pd.DataFrame()
            e_r['ER']=max
            da=e_r['ER']
            new_row = pd.concat([gt,wd,ws,wv,da], axis=1)
            df_all=pd.concat([df_all,pd.DataFrame(new_row, index=[0])])
    elif x=='PREC_거리':
        for i in range(j):
            if i==0:
                da=pd.DataFrame({f'{x[-2:]}_O' : [1], f'{x[-2:]_1' : [0], f'{x[-2:]_2' : [0], f'{x[-2:]_3' : [0], f'{x[-2:]_4' : [0], f'{x[-2:]_5': [0], f'{x[-2:]}_6': [0], f'{x[-2:]_7': [0]})
            elif i==1:
                da=pd.DataFrame({f'{x[-2:]}_O' : [0], f'{x[-2:]_1' : [1], f'{x[-2:]_2' : [0], f'{x[-2:]_3' : [0], f'{x[-2:]_4' : [0], f'{x[-2:]_5': [0], f'{x[-2:]}_6': [0], f'{x[-2:]_7': [0]})
            elif i==2:
                da=pd.DataFrame({f'{x[-2:]}_O' : [0], f'{x[-2:]_1' : [0], f'{x[-2:]_2' : [1], f'{x[-2:]_3' : [0], f'{x[-2:]_4' : [0], f'{x[-2:]_5': [0], f'{x[-2:]}_6': [0], f'{x[-2:]_7': [0]})
            elif i==3:
                da=pd.DataFrame({f'{x[-2:]}_O' : [0], f'{x[-2:]_1' : [0], f'{x[-2:]_2' : [0], f'{x[-2:]_3' : [1], f'{x[-2:]_4' : [0], f'{x[-2:]_5': [0], f'{x[-2:]}_6': [0], f'{x[-2:]_7': [0]})
            elif i==4:
                da=pd.DataFrame({f'{x[-2:]}_O' : [0], f'{x[-2:]_1' : [0], f'{x[-2:]_2' : [0], f'{x[-2:]_3' : [0], f'{x[-2:]_4' : [1], f'{x[-2:]_5': [0], f'{x[-2:]}_6': [0], f'{x[-2:]_7': [0]})
            elif i==5:
                da=pd.DataFrame({f'{x[-2:]}_O' : [0], f'{x[-2:]_1' : [0], f'{x[-2:]_2' : [0], f'{x[-2:]_3' : [0], f'{x[-2:]_4' : [0], f'{x[-2:]_5': [1], f'{x[-2:]}_6': [0], f'{x[-2:]_7': [0]})
            elif i==6:
                da=pd.DataFrame({f'{x[-2:]}_O' : [0], f'{x[-2:]_1' : [0], f'{x[-2:]_2' : [0], f'{x[-2:]_3' : [0], f'{x[-2:]_4' : [0], f'{x[-2:]_5': [0], f'{x[-2:]}_6': [1], f'{x[-2:]_7': [0]})
            elif i==7:
                da=pd.DataFrame({f'{x[-2:]}_O' : [0], f'{x[-2:]_1' : [0], f'{x[-2:]_2' : [0], f'{x[-2:]_3' : [0], f'{x[-2:]_4' : [0], f'{x[-2:]_5': [0], f'{x[-2:]}_6': [0], f'{x[-2:]_7': [1]})
            max = da.idxmax(axis=1)
            e_r=pd.DataFrame()
            e_r['ER']=max
            da=e_r['ER']
            new_row = pd.concat([gt,wd,ws,wv,da], axis=1)
            df_all=pd.concat([df_all,pd.DataFrame(new_row, index=[0])])
    else :
        for i in range(i):
            if i==0:
                da=pd.DataFrame({f'{x[-2:]}_O' : [1], f'{x[-2:]_1' : [0], f'{x[-2:]_2' : [0], f'{x[-2:]_3' : [0], f'{x[-2:]_4' : [0], f'{x[-2:]_5': [0], f'{x[-2:]}_6': [0], f'{x[-2:]_7': [0]})
            elif i==1:
                da=pd.DataFrame({f'{x[-2:]}_O' : [0], f'{x[-2:]_1' : [1], f'{x[-2:]_2' : [0], f'{x[-2:]_3' : [0], f'{x[-2:]_4' : [0], f'{x[-2:]_5': [0], f'{x[-2:]}_6': [0], f'{x[-2:]_7': [0]})
            elif i==2:
                da=pd.DataFrame({f'{x[-2:]}_O' : [0], f'{x[-2:]_1' : [0], f'{x[-2:]_2' : [1], f'{x[-2:]_3' : [0], f'{x[-2:]_4' : [0], f'{x[-2:]_5': [0], f'{x[-2:]}_6': [0], f'{x[-2:]_7': [0]})
            elif i==3:
                da=pd.DataFrame({f'{x[-2:]}_O' : [0], f'{x[-2:]_1' : [0], f'{x[-2:]_2' : [0], f'{x[-2:]_3' : [1], f'{x[-2:]_4' : [0], f'{x[-2:]_5': [0], f'{x[-2:]}_6': [0], f'{x[-2:]_7': [0]})
            elif i==4:
                da=pd.DataFrame({f'{x[-2:]}_O' : [0], f'{x[-2:]_1' : [0], f'{x[-2:]_2' : [0], f'{x[-2:]_3' : [0], f'{x[-2:]_4' : [1], f'{x[-2:]_5': [0], f'{x[-2:]}_6': [0], f'{x[-2:]_7': [0]})
            elif i==5:
                da=pd.DataFrame({f'{x[-2:]}_O' : [0], f'{x[-2:]_1' : [0], f'{x[-2:]_2' : [0], f'{x[-2:]_3' : [0], f'{x[-2:]_4' : [0], f'{x[-2:]_5': [1], f'{x[-2:]}_6': [0], f'{x[-2:]_7': [0]})
            elif i==6:
                da=pd.DataFrame({f'{x[-2:]}_O' : [0], f'{x[-2:]_1' : [0], f'{x[-2:]_2' : [0], f'{x[-2:]_3' : [0], f'{x[-2:]_4' : [0], f'{x[-2:]_5': [0], f'{x[-2:]}_6': [1], f'{x[-2:]_7': [0]})
            elif i==7:
                da=pd.DataFrame({f'{x[-2:]}_O' : [0], f'{x[-2:]_1' : [0], f'{x[-2:]_2' : [0], f'{x[-2:]_3' : [0], f'{x[-2:]_4' : [0], f'{x[-2:]_5': [0], f'{x[-2:]}_6': [0], f'{x[-2:]_7': [1]})
            max = da.idxmax(axis=1)
            e_r=pd.DataFrame()
            e_r['ER']=max
            da=e_r['ER']
            new_row = pd.concat([gt,wd,ws,wv,da], axis=1)
            df_all=pd.concat([df_all,pd.DataFrame(new_row, index=[0])])
    return da
# 시드고정
seed_everything(42)
start = time.time()
# 시작
while(1): 
    key, result = 0, 0
    if __name__ == "__main__":
        key, result = main()
    # 데이터 불러오기
    df_raw = pd.read_csv(f'{result}.csv',index_col=None)
    print('\n\n데이터 증강을 시작합니다.')
    # 데이터 증강
    d_c=df_raw.columns
    ctgan=CTGAN(epochs=50)
    ctgan.fit(df_raw, d_c)
    df_fake=ctgan.sample(1000)
    print(df_fake.describe())
    print(df_raw.describe())
    df_new = pd.concat([df_raw,df_fake], axis=0, ignore_index=True)
    print(df_new.info())
    print(df_new)

    # 증강데이터 사용
    df = df_new
    print(df.describe())

    # 가데이터프레임
    ndf = pd.DataFrame()
    
    # 함형
    df_st = pd.get_dummies(df['함 형'])
    print(df_st.columns)
    # 풍향
    wd = df['풍향'].to_numpy()
    df_wd, wd_list=WD(wd)
    # 풍속
    ws = df['풍속']
    # 파고
    pago = df['파 고']
    # DA
    if key == 1 or key == 3:
        rep = df['거리오차'].to_numpy()
        da, r_list, df_c, cluster_num = REP(rep, key)
        target = df['거리오차']
    else :
        dep = df['편의오차'].to_numpy()
        da, r_list, df_c, cluster_num = DEP(dep, key)
        target = df['편의오차']
    # 학습데이터 생성
    df_a=pd.concat([df_st, ws, df_wd, pago, da, target], axis=1)
    print(df_a.info())
    
    df=df_a.astype(float)
    # 전달정확도 개수 확인
    df['ER']=0
    for x in r_list:
         for i, j in enumerate(df[x]):
            if j == 1.0:
                df['ER'][i]=x
    # 풍향 개수 확인
    df['wd']=0
    for x in wd_list:
         for i, j in enumerate(df[x]):
            if j == 1.0:
                df['wd'][i]=x


    # RAW 데이터와 AUG데이터 분리
    df_raw=df[:203]
    df_aug=df[203:]
    print('풍향 칼럼별 개수')
    print('\n\nraw')
    print(df_raw['wd'].value_counts())
    print('\n\naug')
    print(df_aug['wd'].value_counts())
    # 풍향 개수 맞추기
    df_a = wd_match(key)

    # 데이터 합치기
    df_all=pd.concat([df_a,df_raw], axis=0)
    print('\n\n풍향 칼럼별 총합 개수 확인')
    print(df_all['wd'].value_counts())
    print('\n\n오차범주 확인')
    print(df_all['ER'].value_counts())

    df_all=df_all[df_c]
    df_raw=df_raw[df_c]
    df_all.to_csv(f'{result}_train.csv', encoding='utf-8',index=False)
    df_raw.to_csv(f'{result}_raw.csv', encoding='utf-8',index=False)
    print()
    print(f'\n\n{len(df_all)}개의 전처리가 완료되었습니다.')

    # 회귀시작
    while(1):
        if __name__ == "__main__":
            reg_k = main2()
            if reg_k==3:
                ('\n\n종료합니다.')
                break
        print(f'{result} 회귀를 시작합니다.\n\n')
        print('평가지표를 선택해주세요. :')
        print('1. RMSE')
        print('2. MAE')
        print('3. R2')
        z_list=['rmse', 'mae', 'r2']
        z=int(input('숫자를 입력해주세요. : '))
        

        if reg_k == 1:
            df_reg = pd.read_csv(f'{result}_raw.csv', index_col=None)
            if key == 1 or key == 3:
                reg = TabularPredictor(label='거리오차', eval_metric=z_list[z-1]).fit(df_reg,presets='best_quality', num_bag_folds=8)
                ld_board=reg.leaderboard(df_reg, silent=True)
                print(ld_board)
            elif key == 2 or key == 4:
                reg = TabularPredictor(label='편의오차', eval_metric=z_list[z-1]).fit(df_reg, presets='best_quality', num_bag_folds=8)
                ld_board=reg.leaderboard(df_reg, silent=True)
                print(ld_board)
            print('\n\n성능평가 완료')
        elif reg_k == 2 :
              df_reg = pd.read_csv(f'{result}_train.csv',index_col=None)
              if key == 1 or key == 3:
                reg = TabularPredictor(label='거리오차', eval_metric=z_list[z-1]).fit(df_reg, presets='best_quality', num_bag_folds=8)
                ld_board=reg.leaderboard(df_reg, silent=True)
                print(ld_board)
              elif key == 2 or key == 4:
                reg = TabularPredictor(label='편의오차', eval_metric=z_list[z-1]).fit(df_reg, presets='best_quality', num_bag_folds=8)
                ld_board=reg.leaderboard(df_reg, silent=True)
                print(ld_board)
            print('\n\n성능평가 완료')
        break
    if reg_k==1 or reg_k==3:
         ('\n\n종료합니다.')
         break
    d_all=pd.DataFrame()
    da = pd.DataFrame()
    for i in range(0,5):
        gt=ST(i)
        for j in range(0,3): 
                ws = WS(j)
                for k in range(0,8):
                    wd=WDD(k)
                    for l in range(0,3): 
                        wv=WV(l)
                        da=DA(result, cluster_num)
    predictions= reg.predict(df_all)
    df_all['DA'] = predictions

    gt_1=df_all[['DDH', 'FFG', 'PCC', 'PKG', 'PKM']]
    max = gt_1.idxmax(axis=1)
    df_all['함형'] = max

    wd_1=df_all[['풍향_O', '풍향_1', '풍향_2','풍향_3', '풍향_4', '풍향_5', '풍향_6', '풍향_7']]
    max = wd_1.idxmax(axis=1)
    df_all['풍향']=max
    df_sum = df_all[['함형','풍향', '풍속','파 고', 'ER', 'DA']]
    print(df_sum.info())

    df_sum.to_csv(f'{result}_pred.csv', encoding='utf-8',index=False)
    print(f"{result} 무기효과 산출 완료!")
    break
end = time.time()
print(f"\n\n소요시간: {end-start:0.2f}s")

In [ ]:
• result.py
import pandas as pd 
import numpy as np
# cumulative_frequency 기준
csv_1 = ['MPI_거리_pred.csv','MPI_편의_pred.csv','PREC_거리_pred.csv','PREC_편의_pred.csv']
csv_2 = ['MPI_거리_raw.csv','MPI_편의_raw.csv','PREC_거리_raw.csv','PREC_편의_raw.csv']

st= ['PKM', 'PKG', 'PCC', 'FFG', 'DDH']
ws=[5, 8, 10] 
wv=[0.5, 1.0, 1.5]

# 각각의 전달정확도 담을 튜플
DA_list=[[],[],[],[]]

for c in csv_1:
    mpi_r=pd.read_csv(c, index_col=None)
    for n in range(4): 
        if c==csv_1[n]:
            # raw data 불러오기
            cluster_value = pd.read_csv(csv_2[n], index_col=None)
            for i in st:
                if i == 'DDH' : 
                    for s in ws:
                        for v in wv:
                            val_sum=[]
                            temp_val = 0
                            all_da, da_n=0,0
                            #RAW에서 최빈수 산출
                            temp = cluster_value[(cluster_value[i]==1) & (cluster_value['파 고']==v) & (cluster_value['풍속']==s)]
                            val_counts=temp['da'].value_counts()
                            idx = val_counts.index.tolist()
                            val = val_counts.values.tolist()
                            
                            for j in range(len(val)):
                                temp = mpi_r[(mpi_r['함형']==i) & (mpi_r['파 고']==v) & (mpi_r['풍속']==s) & (mpi_r['da']==idx[j])]
                                temp_val=round(temp['DA'].mean(),2) * val[j]
                                val_sum.append(temp_val)
                            if val != []:
                                all_da=sum(val_sum)
                                da_n=sum(val)
                                mean_da=round(all_da/da_n,3)
                            else :
                                mean_da = np.nan
                                DA_list[n].append(mean_da)
                elif i == 'FFG':
                    for s in ws:
                        for v in wv:
                            val_sum=[]
                            temp_val = 0
                            all_da, da_n=0,0
                            #RAW에서 최빈수 산출
                            temp = cluster_value[(cluster_value[i]==1) & (cluster_value['파 고']==v) & (cluster_value['풍속']==s)]
                            val_counts=temp['da'].value_counts()
                            idx = val_counts.index.tolist()
                            val = val_counts.values.tolist()
                            
                            for j in range(len(val)):
                                temp = mpi_r[(mpi_r['함형']==i) & (mpi_r['파 고']==v) & (mpi_r['풍속']==s) & (mpi_r['da']==idx[j])]
                                temp_val=round(temp['DA'].mean(),2) * val[j]
                                val_sum.append(temp_val)
                            if val != []:
                                all_da=sum(val_sum)
                                da_n=sum(val)
                                mean_da=round(all_da/da_n,3)
                            else :
                                mean_da = np.nan
                                DA_list[n].append(mean_da)
                elif i == 'PCC":
                    for s in ws:
                        for v in wv:
                            val_sum=[]
                            temp_val = 0
                            all_da, da_n=0,0
                            #RAW에서 최빈수 산출
                            temp = cluster_value[(cluster_value[i]==1) & (cluster_value['파 고']==v) & (cluster_value['풍속']==s)]
                            val_counts=temp['da'].value_counts()
                            idx = val_counts.index.tolist()
                            val = val_counts.values.tolist()
                            
                            for j in range(len(val)):
                                temp = mpi_r[(mpi_r['함형']==i) & (mpi_r['파 고']==v) & (mpi_r['풍속']==s) & (mpi_r['da']==idx[j])]
                                temp_val=round(temp['DA'].mean(),2) * val[j]
                                val_sum.append(temp_val)
                            if val != []:
                                all_da=sum(val_sum)
                                da_n=sum(val)
                                mean_da=round(all_da/da_n,3)
                            else :
                                mean_da = np.nan
                                DA_list[n].append(mean_da)

                elif i == 'PKG':
                    for s in ws:
                        for v in wv:
                            val_sum=[]
                            temp_val = 0
                            all_da, da_n=0,0
                            #RAW에서 최빈수 산출
                            temp = cluster_value[(cluster_value[i]==1) & (cluster_value['파 고']==v) & (cluster_value['풍속']==s)]
                            val_counts=temp['da'].value_counts()
                            idx = val_counts.index.tolist()
                            val = val_counts.values.tolist()
                            
                            for j in range(len(val)):
                                temp = mpi_r[(mpi_r['함형']==i) & (mpi_r['파 고']==v) & (mpi_r['풍속']==s) & (mpi_r['da']==idx[j])]
                                temp_val=round(temp['DA'].mean(),2) * val[j]
                                val_sum.append(temp_val)
                            if val != []:
                                all_da=sum(val_sum)
                                da_n=sum(val)
                                mean_da=round(all_da/da_n,3)
                            else :
                                mean_da = np.nan
                                DA_list[n].append(mean_da)
                else :
                    for s in ws:
                        for v in wv:
                            val_sum=[]
                            temp_val = 0
                            all_da, da_n=0,0
                            #RAW에서 최빈수 산출
                            temp = cluster_value[(cluster_value[i]==1) & (cluster_value['파 고']==v) & (cluster_value['풍속']==s)]
                            val_counts=temp['da'].value_counts()
                            idx = val_counts.index.tolist()
                            val = val_counts.values.tolist()
                            
                            for j in range(len(val)):
                                temp = mpi_r[(mpi_r['함형']==i) & (mpi_r['파 고']==v) & (mpi_r['풍속']==s) & (mpi_r['da']==idx[j])]
                                temp_val=round(temp['DA'].mean(),2) * val[j]
                                val_sum.append(temp_val)
                            if val != []:
                                all_da=sum(val_sum)
                                da_n=sum(val)
                                mean_da=round(all_da/da_n,3)
                            else :
                                mean_da = np.nan
                                DA_list[n].append(mean_da)
                                
# 풍속, 파고별 데이터 생성
alle=mpi_r[['함형','풍속','파 고']]
alle=alle.rename(columns={'함형':'S.T',"풍속':'Ws','파 고': 'Wv'})
alle=alle.drop_duplicates(keep='last') 
alle=alle.reset_index(drop=True)
                          
alle['mpi_r']=DA_list[0]
alle['mpi_d']=DA_list[1] 
alle['prec_r']=DA_list[2]
alle['prec_d']=DA_list[3]

# 파고 1 데이터
alle=alle[(alle['Wv']==1.0)]
print(alle.reset_index(drop=True))
alle.to_csv('result.csv', encoding='utf-8', index=False)